In [225]:
%load_ext autoreload
%autoreload
%cd /home/axel/master/ais-visualization/src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/axel/master/ais-visualization/src


In [226]:
from main import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from pathlib import Path

In [227]:
# alt du må gjøre er å endre navnet her og parameterene til funksjonen i bunn, så tror jeg det skal gå
file_pre = "ex_boost_white/logg_dat_SemanticPolitifactW90.json"

In [228]:
def _parse_boosting_info(prop_values: dict, prop_type: str, enum_name: str):
    cols = {}
    for p in prop_values:
        if type(p) == str:
            if len(cols) == 0:
                continue
            else:
                for k, v in cols.items():
                    v.append(pd.NA)
        else:
            memberships: dict = p[enum_name]
            for k, v in memberships.items():
                cur_val = cols.get(k)
                if cur_val is None:
                    cols[k] = [v]
                else:
                    cols[k].append(v)

    df_dict = {}
    for k, v in cols.items():
        # v = np.add.reduceat(v, np.arange(0, len(v), 5))
        df_dict[f"{prop_type}_{k}"] = v
    print(df_dict)
    return (prop_type, df_dict)

In [229]:
def gen_cor_w_noreg_cols(df: pd.DataFrame, key: str):
    n_cor = df[f"{key}_cor"]
    n_wrong = df[f"{key}_wrong"]
    n_no_reg = df[f"{key}_no_reg"]

    df[f"{key}_precision"] = n_cor/(n_cor+n_wrong)
    df[f"{key}_accuracy"] = n_cor/(n_cor+n_wrong+n_no_reg)

In [230]:

with open(f"./../../ais/logg_dat/{file_pre}", "r") as f:
    js_obj = json.load(f)
[(v, i["property_type"]) for i in js_obj["meta_props"] for k, v in enumerate(i) ]

metas = js_obj["meta_props"]

for idx, v in enumerate(metas):
    print(idx, v["property_type"])

boost_accuracy = metas[0]
boost_accuracy_test = metas[1]

(p, d) = _parse_boosting_info(boost_accuracy_test["prop_values"], boost_accuracy_test["property_type"], "MappedInts")
(m, l) = _parse_boosting_info(boost_accuracy["prop_values"], boost_accuracy["property_type"], "MappedInts")
df_dict = []
l.update(d)
ddf = pd.DataFrame(l)

gen_cor_w_noreg_cols(ddf, "BoostAccuracyTest")
gen_cor_w_noreg_cols(ddf, "BoostAccuracy")


0 BoostAccuracy
1 BoostAccuracyTest
2 FoldAccuracy
3 Runtime
{'BoostAccuracyTest_cor': [640, 457, 463, 738, 820, 824, 820, 824, 828, 826], 'BoostAccuracyTest_no_reg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'BoostAccuracyTest_wrong': [449, 632, 626, 351, 269, 265, 269, 265, 261, 263]}
{'BoostAccuracy_cor': [6034, 4015, 4068, 7132, 7642, 7668, 7707, 7730, 7760, 7790], 'BoostAccuracy_no_reg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'BoostAccuracy_wrong': [3858, 5877, 5824, 2760, 2250, 2224, 2185, 2162, 2132, 2102]}


In [231]:
i = 4
step = 5
ddf[i::step]["BoostAccuracy_accuracy"]

4    0.772543
9    0.787505
Name: BoostAccuracy_accuracy, dtype: float64

In [232]:
def get_accs_std(ddf, num_folds: int,  num_boost_rounds: int):
    stds_train = []
    accs_train = []
    stds_test = []
    accs_test = []
    step = num_folds
    for i in range(0, num_boost_rounds):
        stds_train.append(ddf[i::step]["BoostAccuracy_accuracy"].std())
        accs_train.append(ddf[i::step]["BoostAccuracy_accuracy"].mean())
        stds_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].std())
        accs_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].mean())
    return pd.DataFrame.from_dict({"accs_train": accs_train, "stds_train": stds_train, "accs_test": accs_test, "stds_test": stds_test})

In [233]:
def plot_boosting_train_test_acc(ddf: pd.DataFrame, num_folds, num_boost_rounds):
    n = 0
    markers = ["circle", "square", "diamond", "cross", "star", "triangle-up", "triangle-down", "triangle-left", "triangle-right"]

    fig = go.Figure()
    boosts = get_accs_std(ddf, num_folds, num_boost_rounds)
    fig.add_trace(go.Scatter(x=np.arange(0, len(ddf)), y=boosts["accs_test"],
                        mode='lines+markers',
                        name='BoostAccuracyTest',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=boosts["stds_test"],
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    
    n+=1
    fig.add_trace(go.Scatter(x=np.arange(0, len(ddf)), y=boosts["accs_train"],
                        mode='lines+markers',
                        name='BoostAccuracyTrain',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=boosts["stds_test"],
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    n += 1
    

    
    fig.update_layout(height = 500)
    fig.update_xaxes(range=[-0.5,num_boost_rounds - 0.5])
    fig.update_layout(legend_title_text=f" Mean accuracy <br> of {num_boost_rounds} boosting rounds")
    write_image(fig, f"{file_pre}", f"agg_boost.png")

    fig.show()



0    0.587695
Name: BoostAccuracyTest_accuracy, dtype: float64
1    0.419651
Name: BoostAccuracyTest_accuracy, dtype: float64
2    0.425161
Name: BoostAccuracyTest_accuracy, dtype: float64
3    0.677686
Name: BoostAccuracyTest_accuracy, dtype: float64
4    0.752984
Name: BoostAccuracyTest_accuracy, dtype: float64
5    0.756657
Name: BoostAccuracyTest_accuracy, dtype: float64
6    0.752984
Name: BoostAccuracyTest_accuracy, dtype: float64
7    0.756657
Name: BoostAccuracyTest_accuracy, dtype: float64
8    0.760331
Name: BoostAccuracyTest_accuracy, dtype: float64
9    0.758494
Name: BoostAccuracyTest_accuracy, dtype: float64


In [ ]:
plot_boosting_train_test_acc(ddf, 10, 10)